# Домашнее задание по курсу анализа данных # 2
Версия от 14.11.2021

**Автор:** Миленькин Александр

**Выполнил:** Кочетков Дмитрий

# Парсим рейтинг преподавателей с Вики-Физтех

<div class="alert alert-info">
    
Итак, перед вами **VIKI-MIPT** - сборник отзывов о кафедрах и преподавателях МФТИ: ` http://wikimipt.org/wiki/ `

Эта кладезь отзывов не раз давала студентам оценить психологический портрет потенциальных экзаменаторов и преподавателей. 

**Задача** - собрать данные по всем текущим преподавателям всех `институтских кафедр` МФТИ.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# Ссылка на страницу "Кафедры по алфавиту"
departments_list_url = 'http://wikimipt.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%84%D0%B5%D0%B4%D1%80%D1%8B_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83'
driver.get(departments_list_url)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Dmitry Kochetkov\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_25588/740001482.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [2]:
# Ссылки на страницы кафедр
department_urls = []

for link in driver.find_elements(By.XPATH, '//div[@class="mw-category"]//div[@class="mw-category-group"]//ul//li//a'):
    department_urls.append(link.get_property('href'))

In [3]:
# Для обработки оценок

def parse_score(raw_score: str) -> float:
    if raw_score == None or type(raw_score) != str or raw_score == '':
        return None
    if raw_score == '( нет голосов )':
        return 0.0
    return float(raw_score[0:raw_score.find(' ')])

In [4]:
teachers = []

for department_url in department_urls:
    driver.get(department_url)
    department = driver.find_element(By.ID, "firstHeading").text
    
    teacher_urls = []
    for teacher_link in driver.find_elements(By.XPATH, '//div[@id="mw-content-text"]//div[@style="-moz-column-count:3; column-count:3; -webkit-column-count:3"]//ul//li//a'):
        teacher_urls.append(teacher_link.get_property('href'))

    for teacher_link in driver.find_elements(By.XPATH, '//div[@class="gallerytext"]//a'):
        teacher_urls.append(teacher_link.get_property('href'))

    for teacher_url in teacher_urls:
        driver.get(teacher_url)

        full_name = driver.find_element_by_id('firstHeading').text

        try:
            card = driver.find_element(By.ID, "mw-content-text").find_element(By.TAG_NAME, 'table') # Табличка в правой части страницы
            
            rows = card.find_elements_by_tag_name('tr') # Ряды таблички в правой части страницы

            birth_date = None
            alma_mater = None
            degree = None
            knowledge_score = None
            teaching_score = None
            communication_skill = None
            easiness_score = None
            overall_score = None

            # первые два ряда - имя и фотография, их пропускаем

            for row in rows[2:]:
                # row_header = row.find_element_by_tag_name('t')
                try:
                    # пытаемся достать th
                    row_header = row.find_element_by_tag_name('th').text # Пробуем достать header
                    # print(row_header.text)

                    if row_header == 'Дата рождения':
                        birth_date = row.find_element_by_tag_name('td').text
                    if row_header == 'Альма-матер':
                        alma_mater = row.find_element_by_tag_name('td').text
                    elif row_header == 'Учёная степень':
                        degree = row.find_element_by_tag_name('td').text
                    elif '—' in row_header:
                        # для умерших преподавателей могут быть указаны годы жизни внутри тега th. В этом случае дата рождения и дата смерти разделены знаком тире.

                        years_of_live = row.find_element_by_tag_name('th').text
                        birth_date = years_of_live[0:years_of_live.find('—')-1]
                    
                except NoSuchElementException:
                    # Если нет th, то возможно мы попали в оценки. Это элемент table.

                    try:
                        scores_table = row.find_element_by_tag_name('table')
                        for score_table_row in scores_table.find_elements_by_tag_name('tr'):
                            score_name = score_table_row.find_element_by_tag_name('td').text
                            if score_name == 'Знания':
                                knowledge_score = score_table_row.find_element_by_class_name('starrating-avg').text
                                knowledge_score = parse_score(knowledge_score)
                            elif score_name == 'Умение преподавать':
                                teaching_score = score_table_row.find_element_by_class_name('starrating-avg').text
                                teaching_score = parse_score(teaching_score)
                            elif score_name == 'В общении':
                                communication_skill = score_table_row.find_element_by_class_name('starrating-avg').text
                                communication_skill = parse_score(communication_skill)
                            elif score_name == '«Халявность»':
                                easiness_score = score_table_row.find_element_by_class_name('starrating-avg').text
                                easiness_score = parse_score(easiness_score)
                            elif score_name == 'Общая оценка':
                                overall_score = score_table_row.find_element_by_class_name('starrating-avg').text
                                overall_score = parse_score(overall_score)
                    
                    except NoSuchElementException:
                        break
            
            teachers.append([full_name, birth_date, alma_mater, department, degree, knowledge_score, teaching_score, communication_skill, easiness_score, overall_score])
        except NoSuchElementException:
            print('Warning! No data found for teacher {}'.format(full_name))

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_25588/2676479202.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  full_name = driver.find_element_by_id('firstHeading').text
D:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
D:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
D:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:446: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* comman

Warning! No data found for teacher Плоткин Арнольд Леонидович


In [5]:
import pandas as pd
teachers = pd.DataFrame(teachers, columns=['full_name', 'birth_date', 'alma_mater', 'department', 'degree', 'knowledge_score', 'teaching_score', 'communication_skill', 'easiness_score', 'overall_score'], dtype=str)
teachers.head()

,full_name,birth_date,alma_mater,department,degree,knowledge_score,teaching_score,communication_skill,easiness_score,overall_score
0,Иванов Юрий Григорьевич,None,None,Военная кафедра,None,5.0,5.0,5.0,5.0,5.0
1,Кваченко Андрей Владимирович,None,Тамбовское ВВАУЛ им. М. М. Расковой,Военная кафедра,кандидат технических наук,3.5,3.0,2.5,3.5,3.5
2,Корнеев Виктор Александрович,None,None,Военная кафедра,None,5.0,5.0,5.0,4.0,5.0
3,Летуновский Сергей Владимирович,None,None,Военная кафедра,None,3.0,3.0,3.0,2.0,3.0
4,Пешев Андрей Леонидович,None,None,Военная кафедра,None,0.0,0.0,0.0,0.0,0.0


In [6]:
teachers.to_csv('teachers.csv')

In [7]:
teachers.dtypes

full_name              object
birth_date             object
alma_mater             object
department             object
degree                 object
knowledge_score        object
teaching_score         object
communication_skill    object
easiness_score         object
overall_score          object
dtype: object